In [2]:
#가상환경 cuda로 실행할 것

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os

In [2]:
import torch
torch.__version__

'1.12.1+cpu'

In [3]:
y_train=[]
for i in range(20):
    y_train+=[i]*150

y_test=[]
for i in range(20):
    y_test+=[i]*50

In [4]:
from torch.utils.data import Dataset
import numpy as np

class TextCuboidDataset(Dataset):
    def __init__(self, data_dir, labels):
        self.data_dir = data_dir
        self.labels = labels
        self.data = []
        
        for i in range(3000):
            data_path = data_dir+'/cuboid%d.npy'%i
            self.data.append(np.load(data_path))
            
#         print (np.asarray(self.data).shape)
#         print (np.asarray(self.label).shape)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [5]:
from torch.utils.data import Dataset
import numpy as np

class TestDataset(Dataset):
    def __init__(self, data_dir, labels):
        self.data_dir = data_dir
        self.labels = labels
        self.data = []
        
        for i in range(1000):
            data_path = data_dir+'/cuboid_test%d.npy'%i
            self.data.append(np.load(data_path))
            
#         print (np.asarray(self.data).shape)
#         print (np.asarray(self.label).shape)
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [6]:
data_dir = 'D:/textcuboid_20ng_multi'
dataset = TextCuboidDataset(data_dir, y_train)

In [7]:
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [8]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        filter_cnt = 1024
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=filter_cnt, kernel_size=(1, 1024))
        self.relu = nn.ReLU()
        self.pooling = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(filter_cnt, 512)
        self.fc2 = nn.Linear(512, 128)
        self.output_layer = nn.Linear(128, 20)

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = x.view(-1,x.shape[1],x.shape[2]*x.shape[3])
        x = self.pooling(x).view(-1,x.shape[1])
        
        x = self.fc1(x)
        x = self.relu(x)

        x = self.fc2(x)
        x = self.relu(x)

        out=self.output_layer(x)
        return self.output_layer(x)

model=CNN().cuda()

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torchsummary

In [ ]:
torchsummary.summary(model,input_size=(3,2000,1024))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
total_batch = len(dataloader)
print('총 배치의 수 : {}'.format(total_batch))

In [ ]:
import torch.optim as optim
from torch.nn import CrossEntropyLoss

criterion = CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters())

num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, (data,label) in enumerate(dataloader):
        data = data.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.long)
        
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        _, predicted = torch.max(outputs.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()

    print(f'Epoch: {epoch+1}/{num_epochs} Loss: {running_loss/len(dataloader)} Accuracy: {100 * correct / total}')

In [ ]:
torch.save(model.state_dict(), 'D:/save model/cnn_model.pt')

In [10]:
model.load_state_dict(torch.load('D:/save model/cnn_model.pt'))

<All keys matched successfully>

In [11]:
test_dir = 'D:/textcuboid_20ng_multi_test'
testset = TestDataset(test_dir, y_test)
testloader = DataLoader(testset, batch_size=128)

In [12]:
labels = []
predicteds = []

model.eval ()

with torch.no_grad ():
    for i, (data, label) in enumerate (testloader):
        
        data = data.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.long)
        
        output = model (data)
        
        _, predicted = torch.max (output, 1)
        accuracy = (label==predicted.squeeze()).float().mean()
        
        labels.extend (label.cpu())
        predicteds.extend (predicted.squeeze().cpu())
        
        print ('accuracy: {:.4f}'.format (accuracy.item()))

accuracy: 0.5781
accuracy: 0.7266
accuracy: 0.8125
accuracy: 0.7266
accuracy: 0.6797
accuracy: 0.7344
accuracy: 0.6562
accuracy: 0.4808


In [13]:
from sklearn.metrics import f1_score
from sklearn import metrics

print(metrics.classification_report(labels, predicteds, digits=3))

              precision    recall  f1-score   support

           0      0.469     0.460     0.465        50
           1      0.674     0.620     0.646        50
           2      0.660     0.660     0.660        50
           3      0.633     0.760     0.691        50
           4      0.765     0.780     0.772        50
           5      0.725     0.740     0.733        50
           6      0.833     0.900     0.865        50
           7      0.755     0.740     0.747        50
           8      0.614     0.700     0.654        50
           9      0.902     0.740     0.813        50
          10      0.863     0.880     0.871        50
          11      0.829     0.580     0.682        50
          12      0.634     0.520     0.571        50
          13      0.820     0.820     0.820        50
          14      0.651     0.820     0.726        50
          15      0.646     0.620     0.633        50
          16      0.483     0.580     0.527        50
          17      0.771    

In [14]:
del model
torch.cuda.empty_cache()